# Word sense disambiguation with Simplified Lesk for Ukrainian

## Senses

Send requests to http://sum.in.ua/ to collect senses for a word.

In [1]:
from lxml import html
import requests

In [2]:
# http://sum.in.ua/ uses custom transliteration

conversion = {
    u'\u0410' : 'A',    u'\u0430' : 'a',
    u'\u0411' : 'B',    u'\u0431' : 'b',
    u'\u0412' : 'V',    u'\u0432' : 'v',
    u'\u0413' : 'Gh',    u'\u0433' : 'gh',
    u'\u0490' : 'G',    u'\u0491' : 'g',
    u'\u0414' : 'D',    u'\u0434' : 'd',
    u'\u0415' : 'E',    u'\u0435' : 'e',
    u'\u0404' : 'Ye',   u'\u0454' : 'je',
    u'\u0416' : 'Zh',   u'\u0436' : 'zh',
    u'\u0417' : 'Z',    u'\u0437' : 'z',
    u'\u0418' : 'Y',    u'\u0438' : 'y',
    u'\u0406' : 'I',    u'\u0456' : 'i',
    u'\u0407' : 'Ji',   u'\u0457' : 'ji',
    u'\u0419' : 'J',    u'\u0439' : 'j',
    u'\u041a' : 'K',    u'\u043a' : 'k',
    u'\u041b' : 'L',    u'\u043b' : 'l',
    u'\u041c' : 'M',    u'\u043c' : 'm',
    u'\u041d' : 'N',    u'\u043d' : 'n',
    u'\u041e' : 'O',    u'\u043e' : 'o',
    u'\u041f' : 'P',    u'\u043f' : 'p',
    u'\u0420' : 'R',    u'\u0440' : 'r',
    u'\u0421' : 'S',    u'\u0441' : 's',
    u'\u0422' : 'T',    u'\u0442' : 't',
    u'\u0423' : 'U',    u'\u0443' : 'u',
    u'\u0424' : 'F',    u'\u0444' : 'f',
    u'\u0425' : 'Kh',    u'\u0445' : 'kh',
    u'\u0426' : 'C',   u'\u0446' : 'c',
    u'\u0427' : 'Ch',   u'\u0447' : 'ch',
    u'\u0428' : 'Sh',   u'\u0448' : 'sh',
    u'\u0429' : 'Shh',  u'\u0449' : 'shh',
    u'\u044c' : 'j',
    u'\u042e' : 'Ju',   u'\u044e' : 'ju',
    u'\u042f' : 'Ja',   u'\u044f' : 'ja',
    u'\'' : '.'
}

def transliterate(word):
    translit = []
    for c in word:
        translit.append(conversion.get(c, c))
    return ''.join(translit)

In [3]:
print(transliterate("серце"))
print(transliterate("знущається"))
print(transliterate("бур'ян"))

serce
znushhajetjsja
bur.jan


In [4]:
URL = "http://sum.in.ua/s/"

def get_senses(word):
    """
    Extract senses for WORD from the dictionary.
    """
    word = transliterate(word)
    page = requests.get(URL + word)
    tree = html.fromstring(page.text)
    senses = tree.xpath("//p[@class='znach']")
    print(len(senses), "senses found.")
    return list(enumerate([" ".join(i.xpath('text()')) for i in senses], 1))

In [5]:
word = "серце"
senses = get_senses(word)
print(senses[5][1])

7 senses found.
   Металева, циліндричної форми частина
дзвона, розташована у внутрішній його частині; язик.
Стара дзвіниця.. Від серця дзвона падав вірьовка. Місячна
 ніч. Церковний сторож відбивав години  ; Він хотів ще потягнути мотуза, напружився і таки;
потягнув, але важке металеве серце ледве дійшло до
обідка дзвона  .



## Collect sentenses from the brown-uk corpus

Read files from https://github.com/brown-uk/corpus/tree/master/data/good. Find all sentences that contain the word in question.

Use https://github.com/lang-uk/tokenize-uk for tokenization:
pip install tokenize_uk

and https://github.com/kmike/pymorphy2 for lemmatization:

pip install -U https://github.com/kmike/pymorphy2/archive/master.zip#egg=pymorphy2

pip install -U pymorphy2-dicts-uk

In [12]:
import os
import re
import pymorphy2
import tokenize_uk
import pymorphy2_dicts_uk
CORPUS_PATH = "./corpus/data/good/"
morph = pymorphy2.MorphAnalyzer(lang='uk')

In [13]:
def find_sentences(text, word):
    """
    From TEXT select sentences that contain WORD.
    """
    sentences_w_word = []
    paragraphs = tokenize_uk.tokenize_text(text)
    for paragraph in paragraphs:
        for sentence in paragraph:
            for token in sentence:
                if morph.parse(token)[0].normal_form == word:
                    sentences_w_word.append(sentence)
                    break
    return sentences_w_word

In [14]:
data = []

for filename in os.listdir(CORPUS_PATH):
    with open(CORPUS_PATH + filename, "r") as f:
        text = f.read()
    body = text[text.find("<body>"):]
    body = re.sub(r"<[^<>]+>", "", body)
    data += find_sentences(body, word)

print(len(data))

202


In [15]:
data[0]

['Музика',
 'Баха',
 ',',
 'Моцарта',
 ',',
 'Гайдна',
 'завжди',
 'пробуджує',
 'у',
 'серці',
 'світлу',
 'радість',
 '.']

## Implement Simplified Lesk

### 1. Collect context words from senses

Tokenize, downcase, lemmatize, and remove stop-words.

In [16]:
def collect_bow(text):
    """
    Tokenize TEXT with tokenize_uk, lemmatize TEXT with pymorphy2,
    and remove functional words.
    """
    tokens = tokenize_uk.tokenize_words(text)
    bow = []
    for token in tokens:
        parse = morph.parse(token)[0]
        if parse.tag.POS not in {'NUMR', 'NPRO', 'PREP', 'CONJ', 'PRCL'} and \
            not 'PNCT' in parse.tag and \
            not token.isdigit() and \
            token not in ['до', 'на', 'за', 'його']:
            bow.append(parse.normal_form)
    return set(bow)

In [17]:
collect_bow(senses[5][1])

{'важкий',
 'внутрішній',
 'відбивати',
 'вірьовка',
 'година',
 'дзвін',
 'дзвіниця',
 'дійти',
 'металевий',
 'мотуз',
 'місячний',
 'напружитися',
 'ніч',
 'обідок',
 'падати',
 'потягнути',
 'розташований',
 'серце',
 'старий',
 'сторож',
 'форма',
 'хотіти',
 'церковний',
 'циліндричний',
 'частина',
 'язик'}

In [18]:
collect_bow(data[0])

{'бах',
 'гайдн',
 'моцарт',
 'музика',
 'пробуджувати',
 'радість',
 'світлий',
 'серце'}

### 2. Define the best sense for the word

Find the sense that has the biggest overlap with the word context. Use sense 1 as default.

In [19]:
signatures = [(idx, collect_bow(sense)) for idx, sense in senses]

def classify(sentence, word, senses, log=False):
    """
    Find the sense that has the biggest overlap with the word context.
    Use sense 1 as default.
    """
    context = collect_bow(sentence)
    best_sense = 1
    max_overlap = 0
    max_overlap_words = []
    for sense_id, signature in signatures:
        overlap = context.intersection(signature)
        if len(overlap) > max_overlap:
            max_overlap_words = overlap
            max_overlap = len(overlap)
            best_sense = sense_id
    if log:
        print(" ".join(sentence))
        print("Best sense:", best_sense)
        print("{} words overlap: {}.\n".format(max_overlap, ", ".join(max_overlap_words)))
    return best_sense

In [20]:
classify(data[0], word, signatures, log=True)

Музика Баха , Моцарта , Гайдна завжди пробуджує у серці світлу радість .
Best sense: 2
3 words overlap: серце, музика, радість.



2

In [21]:
classify(data[1], word, signatures, log=True)

Чиє ж серце не звеселиться , коли бачиш палке кохання ? Майка й досі Коробова згадує , бо він один клявся кохати її вічно .
Best sense: 2
6 words overlap: кохати, серце, бачити, згадувати, палкий, кохання.



2

In [22]:
classify(data[2], word, signatures, log=True)

Футболки з надписом « Вільна Україна » , значки , надувні кульки й серця , рожеві поролонові носи , і вуха , і ріжки ...
Best sense: 2
5 words overlap: серце, україна, носити, рожевий, вільний.



2

In [23]:
classify(data[90], word, signatures, log=True)

Протягом двох днів в Ужгороді та Мукачеві фахівці Національного інституту серцево - судинної хірургії ім . М . Амосова консультували маленьких закарпатців , які мають хвороби серця .
Best sense: 4
3 words overlap: дніти, серце, хвороба.



4

In [24]:
classify(data[130], word, signatures, log=True)

Вероніка серцем доньки зрозуміла , як важко зараз матері .
Best sense: 2
4 words overlap: зараза, серце, мати, важко.



2

In [25]:
classify(data[157], word, signatures, log=True)

Серце людини приблизно таке ж за розміром , як її кулак .
Best sense: 1
2 words overlap: серце, людина.



1

In [26]:
from collections import Counter

sense_counts = []
for i in range(len(data)):
    sense_id = classify(data[i], word, signatures)
    sense_counts.append(sense_id)
Counter(sense_counts)

Counter({2: 162, 1: 38, 4: 1, 5: 1})

## Your task

1. Choose a word (or a list of words).
2. Collect real-world sentences for the word(s).
3. Implement Simplified Lesk for disambiguating the word(s).
4. Set aside 50 sentences as a test set and manually fix the disambiguation errors that Lesk made. Calculate the quality of WSD by Lesk.
5. Improve the quality of WSD by:
- taking into account word count
- counting IDF score (log of total number of senses divided by the number of senses where the word was used)
- extending the word lists for senses in a semi-supervised manner (using the context of words that were disambiguated with high confidence)

In [42]:
word = "масло"
senses = get_senses(word)
print(senses[2])

3 senses found.
(3, "   Те саме, що   1. Змащувальною й\nзв'язувальною рідиною звичайно є.. газолін, машинне\nмасло та ін.  ;\nВ іншому кутку кімнати змащувала [Жукова] маслом\nінструменти  .\n    і т. ін.  — легко,\nдуже добре, без перешкод удаватися, виходити. Богу\nдякувати, все нам як з маслом іде! За півгодини і зложили\n[січкарню]  ; — Подзвонив я і вдруге\nпрямо на квартиру до Сагайдака, коли кахлів не давали.\nСагайдак поговорив, і діло пішло як по маслу  .")


In [83]:
test_sentences = [
    "Вбігає зразу п'ять здоровенних парубків, вони несуть мішки, повні масла, а в зубах у кожного стирчить по цілій пачці.",
    "Зверху, прямо на маслі, лежить іще один парубок, він блює маслом вниз.",
    "І з тобою, діду, таке буде, якщо багацько масла з'їси.",
    "Будеш піддакувати, ми і тобі голову в масло засунемо.",
    "За ними з куском масла в зубах повзе парубок в цегейковій шапці.",
    "Дівчата, задеріть спідниці, масла дам (регоче).",
    "Одна з виховательок мітко жбурля чималий шматок масла йому в голову.",
    "Входить дебела жінка з вусами, бородавками і куснем масла під пахвою.",
    "Арон, збирайся додому, сьогодні я тобі на вечерю гарненько засмажу шматочок масла.",
    "Він помер, дай йому трохи масла, він його любив.",
    "Опасиста жінка відрізає шматок масла і кладе в зуби мертвому парубку.",
    "Сідаючи в двері, дебела жінка бере Арона під пахву і разом з Ароном і маслом сідає в трамвай.",
    "Трамвай, подзенькуючи, повільно суне в блакитну далечінь під натхненні звуки полонеза Огінського і здавлені крики любителів масла.",
    "Кахикнув і промовив: соняшник, масло і стиглий урюк, хай буде жовтим дурний цей пацюк.",
    "Фадж намастив собі булочку маслом і підсунув тарілку Гаррі.",
    "Герміоно, ти скоро заспіваєш іншої, - попередив Фред, густо намащуючи булочку маслом.",
    "Нарешті залили воду, бензин, масло.",
    "І після всього цього з'являються в замок ці огидні нинішні американці, нав'язують йому машинне масло і кидають в нього подушками!",
    "Після того, як я десять разів видихнув і двадцять разів вилаявся, робота пішла як по маслу!",
    "Підлив масла в вогонь, спалив хату і сарай.",
    "У горах хтось кинув в суп маргарину і масла, звісно ж, змити це холодною водою не вийшло.",
    "Мені потрібно поміняти масло в двигуні.",
    "Кажуть, що в двигун треба лити синтетичні масла.",
    "Пляма масла утворилася на поверхні води.",
    "Викинутий на берег, мертвий дельфін був весь в маслі.",
    "Прокинувся я лиш на другий день і побачив, що стара карга втирала масла в глибокі рани, що утворилися в мене на руці і плечі від ремнів, якими я був прив'язаний.",
    "Світло падало нерівно, плівка масла через це виблискувала.",
    "Світло лампи почало тьм'яніти, бо майже все масло вигоріло.",
    "З цієї ферми ми отримували масло, фрукти і всяку всячину.",
    "Кажуть, що масло утворює на стінці шлунка плівку, тому алкоголь й не засвоюється.",
    "Серцевина горіха теж щедра на масло.",
    "Він змастив маслом двері в стайню.",
    "Ми торгували маслами, тютюном та опіумом.",
    "В гаражі не було ні машинного масла, ні солідолу.",
    "Запах свіжої фарби та машинного масла був хорошим знаком для танкіста.",
    "Тоді крали все, і в тих, хто найбільше крав все йшло, як по маслу.",
    "Щоб механізм працював довго й надійно, його потрібно змащувати маслом.",
    "Оля стояла на кухні, намагаючись відмити руки від масла, яким змащувала кулемет.",
    "Я купив маленький шматочок масла, бо не хотів, щоб воно зіпсувалося.",
    "Цей покидьок напхав мені в стаканчик від морозива масла!",
    "Масло розлили на підлогу, тому Степан і впав.",
    "Поки на аеродромі вкручували новий гвинт, технік змащував кріплення смішшю графіту та масла.",
    "Я змащую дерево на грифі гітари маслами, щоб воно кразе зберігалося.",
    "При консервуванні техніки масло та солідол використовуються для попередження корозії.",
    "Можна виготовати гідроциліндр, який працює на машинному маслі.",
    "Він замінив масло в амортизаторах перед гонкою, бо знав, що воно втікає через мікротріщини.",
    "Господи, та я просто хочу хліба з маслом з'їсти а не оцього всього.",
    "На підозі були уламки бетону, плями масла і кров.",
    "Випари рослинних масел виїдали очі, дихати було просто неможливо.",
    "Техніки вирішили утилізувати масло шляхом спалення, в результаті повернулися додому трішки копченими від густого чорного диму, який утворився."
]
print(len(test_sentences))


50


In [82]:
senses = get_senses(word)
for s in senses:
    print(s[1])
    print("\n==============\n")

3 senses found.
 Харчовий продукт, який
виробляють збиванням вершків або сметани. Заваривши
кашу, не жалуй масла  ; Вона..
підносила до рота шматок хліба, густо змазаний
маслом  ; Відколи Степан Кушнір
отримав поміщицьку тяжкотілу корову, то щоразу
приносив на комнезамські збори загорнуту в ганчірку
грудочку масла  ;   
Катерина від насіння взяла жменю бобу, зварила в горняті,
а він розбух, став м'який, як масло  .
    і т. ін.  — жити в достатках,
розкошах. [Возний:] До кого ж ласкава ся доля лукава.
Такий живе як сир в маслі  ; — В
городі тільки й жити! .. Та я як.. на місці була —
як сир у маслі купалася...  ; Жив
собі як вареник у маслі  ;   — уживається для
вираження незадоволення ким-, чим-небудь. [Лицар:]
Кажи мені по-людськи, що тобі від мене треба? Нащо
рятувала мене з темниці, щоб держати тут?
[Служебка:] Цур дурня й масла грудка!  ;   і т. ін.  —
про те, що дає велике задоволення.



   Жирова речовина, яку видобувають із
мінеральних речовин. Мінеральні масла являють собою
п

In [104]:
correct_senses = [
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    2,
    2,
    3,
    2,
    1,
    2,
    2,
    2,
    2,
    3,
    2,
    2,
    1,
    1,
    2,
    3,
    2,
    2,
    2,
    3,
    3,
    3,
    1,
    1,
    2,
    3,
    3,
    2,
    2,
    2,
    1,
    2,
    3,
    2
]

In [99]:
data = []

for filename in os.listdir(CORPUS_PATH):
    with open(CORPUS_PATH + filename, "r") as f:
        text = f.read()
    body = text[text.find("<body>"):]
    body = re.sub(r"<[^<>]+>", "", body)
    data += find_sentences(body, word)

print(len(data))

20


In [85]:
signatures = [(idx, collect_bow(sense)) for idx, sense in senses]

print(signatures)

[(1, {'сметана', 'достаток', 'місце', 'людськи', 'вареник', 'ласкавий', 'масло', 'жменя', 'город', 'підносити', 'кушнір', 'шматок', 'жити', 'грудка', 'цур', 'катерина', 'лицар', 'вираження', 'виробляти', 'густо', 'приносити', 'загорнутий', 'лукавий', 'сир', 'задоволення', 'тяжкотілий', 'ганчірка', 'незадоволення', 'тобі', 'булий', 'розкіш', 'степан', 'возний', 'доля', 'соб', 'боб', 'т', 'темниця', 'ін', 'насіння', 'харчовий', 'поміщицький', 'розбух', 'служебка', 'комнезамський', 'великий', 'рота', 'до', 'рятувати', 'жалувати', 'грудочка', 'горня', 'вершок', 'хліб', 'взяти', 'мен', 'продукт', 'збивання', 'треба', 'нащо', 'давати', 'коровий', 'казати', 'заваривши', 'живий', 'купатися', 'щоразу', 'став', 'уживатися', 'небути', 'про', 'отримати', 'збора', 'каша', 'змазаний', 'зварити', 'дурень', 'держати', "м'який"}), (2, {'певний', 'гукати', 'масло', 'правильно', 'хазяї', 'перегонка', 'важкий', 'видобувати', 'посилювати', 'смолистий', 'почуття', 'жировий', 'місцевий', 'вугілля', 'може', '

In [86]:
for d in data:
    classify(d, word, signatures, log=True)

Дрібно посічені яблука , злегка протушковані на маслі і поцукровані .
Best sense: 1
1 words overlap: масло.

Готові пироги мастимо маслом , смальцем , олією , смаженою цибулею , подаємо зі сметаною або кислим молоком .
Best sense: 1
2 words overlap: масло, сметана.

А коли молоко було солодке , то смачними вдавалися сир , сметана і масло .
Best sense: 1
3 words overlap: масло, сметана, сир.

В 1962 році , коли він командував радянськими військами , трапилася сумнозвісна подія в Новочеркаську – придушення виступів робітників , які просили хліба та масла у радянської держави .
Best sense: 1
2 words overlap: хліб, масло.

Не так і багато , але були ще рибні консерви : тунець у воді та в маслі .
Best sense: 3
2 words overlap: бути, масло.

10 столових ложок масла кладемо у ринку і розтоплюємо .
Best sense: 1
1 words overlap: масло.

Маслом намащуємо форму чи кілька невеликих форм і , наповнивши їх до половини сумішшю , ставимо у достатньо гарячу братруру .
Best sense: 1
1 words overlap: ма

In [103]:
results = []
for d in test_sentences:
    s = tokenize_uk.tokenize_text(d)[0][0]
    sense = classify(s, word, signatures, log=True)
    results.append(sense)

Вбігає зразу п'ять здоровенних парубків , вони несуть мішки , повні масла , а в зубах у кожного стирчить по цілій пачці .
Best sense: 1
1 words overlap: масло.

Зверху , прямо на маслі , лежить іще один парубок , він блює маслом вниз .
Best sense: 3
2 words overlap: прямо, масло.

І з тобою , діду , таке буде , якщо багацько масла з'їси .
Best sense: 3
2 words overlap: бути, масло.

Будеш піддакувати , ми і тобі голову в масло засунемо .
Best sense: 1
2 words overlap: масло, тобі.

За ними з куском масла в зубах повзе парубок в цегейковій шапці .
Best sense: 3
2 words overlap: масло, за.

Дівчата , задеріть спідниці , масла дам ( регоче ) .
Best sense: 1
1 words overlap: масло.

Одна з виховательок мітко жбурля чималий шматок масла йому в голову .
Best sense: 1
2 words overlap: масло, шматок.

Входить дебела жінка з вусами , бородавками і куснем масла під пахвою .
Best sense: 1
1 words overlap: масло.

Арон , збирайся додому , сьогодні я тобі на вечерю гарненько засмажу шматочок масла 

In [89]:
print(results)

[1, 3, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 3, 3, 1, 1, 1, 3, 3, 2, 3, 1, 1, 2, 3, 1]


In [105]:
correct = 0
for i in range(50):
    if results[i] == correct_senses[i]:
        correct += 1
    else:
        print(test_sentences[i])
        print("\nFound sense:", results[i])
        print("\nCorrect sense:", correct_senses[i])
        print("\n============")
correct = correct*2
print(correct)

Зверху, прямо на маслі, лежить іще один парубок, він блює маслом вниз.

Found sense: 3

Correct sense: 1

І з тобою, діду, таке буде, якщо багацько масла з'їси.

Found sense: 3

Correct sense: 1

За ними з куском масла в зубах повзе парубок в цегейковій шапці.

Found sense: 3

Correct sense: 1

Кахикнув і промовив: соняшник, масло і стиглий урюк, хай буде жовтим дурний цей пацюк.

Found sense: 3

Correct sense: 1

Нарешті залили воду, бензин, масло.

Found sense: 1

Correct sense: 2

І після всього цього з'являються в замок ці огидні нинішні американці, нав'язують йому машинне масло і кидають в нього подушками!

Found sense: 3

Correct sense: 2

Підлив масла в вогонь, спалив хату і сарай.

Found sense: 1

Correct sense: 2

Мені потрібно поміняти масло в двигуні.

Found sense: 1

Correct sense: 2

Кажуть, що в двигун треба лити синтетичні масла.

Found sense: 1

Correct sense: 2

Пляма масла утворилася на поверхні води.

Found sense: 1

Correct sense: 2

Викинутий на берег, мертвий дель

In [113]:
print(collect_bow("Оля мастила маслом масло масла масляний ніж краще б масло"))

{'мастило', 'масляний', 'масло', 'краще', 'оля'}


In [118]:
def classify_wc(sentence, word, senses, log=False):
    """
    Find the sense that has the biggest overlap with the word context.
    Use sense 1 as default.
    """
    context = collect_bow(sentence)
    best_sense = 1
    max_overlap = 0
    max_overlap_words = []
    for sense_id, signature in signatures:
        overlap = context.intersection(signature)
        counter = 0
        sent = []
        for s in sentence:
            parse = morph.parse(s)[0]
            sent.append(parse.normal_form)
        for w in overlap:
            counter += sent.count(w)
        if counter > max_overlap:
            max_overlap_words = overlap
            max_overlap = counter
            best_sense = sense_id
    if log:
        print(" ".join(sentence))
        print("Best sense:", best_sense)
        print("{} words overlap: {}.\n".format(max_overlap, ", ".join(max_overlap_words)))
    return best_sense

In [119]:
results = []
for d in test_sentences:
    s = tokenize_uk.tokenize_text(d)[0][0]
    sense = classify_wc(s, word, signatures, log=True)
    results.append(sense)
    
correct = 0
for i in range(50):
    if results[i] == correct_senses[i]:
        correct += 1
    else:
        print(test_sentences[i])
        print("\nFound sense:", results[i])
        print("\nCorrect sense:", correct_senses[i])
        print("\n============")
correct = correct*2
print(correct)

Вбігає зразу п'ять здоровенних парубків , вони несуть мішки , повні масла , а в зубах у кожного стирчить по цілій пачці .
Best sense: 1
1 words overlap: масло.

Зверху , прямо на маслі , лежить іще один парубок , він блює маслом вниз .
Best sense: 3
3 words overlap: прямо, масло.

І з тобою , діду , таке буде , якщо багацько масла з'їси .
Best sense: 3
2 words overlap: бути, масло.

Будеш піддакувати , ми і тобі голову в масло засунемо .
Best sense: 1
2 words overlap: масло, тобі.

За ними з куском масла в зубах повзе парубок в цегейковій шапці .
Best sense: 3
2 words overlap: масло, за.

Дівчата , задеріть спідниці , масла дам ( регоче ) .
Best sense: 1
1 words overlap: масло.

Одна з виховательок мітко жбурля чималий шматок масла йому в голову .
Best sense: 1
2 words overlap: масло, шматок.

Входить дебела жінка з вусами , бородавками і куснем масла під пахвою .
Best sense: 1
1 words overlap: масло.

Арон , збирайся додому , сьогодні я тобі на вечерю гарненько засмажу шматочок масла 

In [145]:
def computeIDF(sentences):
    sentences_normal = []
    for sentence in sentences:
        sent = []
        for s in sentence:
            parse = morph.parse(s)[0]
            sent.append(parse.normal_form)
        sentences_normal.append(sent)    
    idfDict = {}
    for i in range(50):
        for w in sentences_normal[i]:
            if w in idfDict.keys():
                continue
            else:
                idfDict[w] = []
                for idx, j in enumerate(sentences_normal):
                    if j.count(w) > 0:
                        idfDict[w].append(correct_senses[idx])
    for key in idfDict.keys():
        idfDict[key] = len(set(idfDict[key]))
    return idfDict

In [146]:
tokenized = []
for d in test_sentences:
    tokenized.append(tokenize_uk.tokenize_text(d)[0][0])
idfDict = computeIDF(tokenized)
print(idfDict)

{'вбігати': 1, 'зраза': 1, "п'ять": 1, 'здоровенний': 1, 'парубок': 1, ',': 3, 'вони': 1, 'нести': 1, 'мішок': 1, 'повня': 1, 'масло': 3, 'а': 1, 'в': 3, 'зуб': 1, 'у': 1, 'кожний': 1, 'стирчати': 1, 'по': 2, 'цілий': 1, 'пачка': 1, '.': 3, 'зверху': 1, 'прямо': 1, 'на': 3, 'лежати': 1, 'іще': 1, 'один': 1, 'він': 3, 'блювати': 1, 'вниз': 1, 'і': 3, 'з': 1, 'ти': 1, 'дід': 1, 'такий': 1, 'бути': 3, 'якщо': 1, 'багацько': 1, "з'їсти": 1, 'піддакувати': 1, 'ми': 2, 'тобі': 1, 'голова': 1, 'засунути': 1, 'за': 1, 'кусок': 1, 'повзти': 1, 'цегейковий': 1, 'шапка': 1, 'дівча': 1, 'задрати': 1, 'спідниця': 1, 'дати': 1, '(': 1, 'реготати': 1, ')': 1, 'вихователька': 1, 'мітка': 1, 'жбурля': 1, 'чималий': 1, 'шматок': 1, 'входити': 1, 'дебелий': 1, 'жінка': 1, 'вус': 1, 'бородавка': 1, 'куснути': 1, 'під': 1, 'пахва': 1, 'арон': 1, 'збиратися': 1, 'додому': 2, 'сьогодні': 1, 'я': 2, 'вечеря': 1, 'гарненько': 1, 'засмажити': 1, 'шматочок': 1, 'померти': 1, 'трохи': 1, 'йога': 2, 'любити': 1, '

In [142]:
results = []
for d in test_sentences:
    s = tokenize_uk.tokenize_text(d)[0][0]
    sense = classify_wc(s, word, signatures, log=True)
    results.append(sense)

Зверху, прямо на маслі, лежить іще один парубок, він блює маслом вниз.


In [147]:
def classify_wc_idf(sentence, word, senses, log=False):
    """
    Find the sense that has the biggest overlap with the word context.
    Use sense 1 as default.
    """
    context = collect_bow(sentence)
    best_sense = 1
    max_overlap = 0
    max_overlap_words = []
    for sense_id, signature in signatures:
        overlap = context.intersection(signature)
        counter = 0
        sent = []
        for s in sentence:
            parse = morph.parse(s)[0]
            sent.append(parse.normal_form)
        for w in overlap:
            counter += sent.count(w) / idfDict[w]
        if counter > max_overlap:
            max_overlap_words = overlap
            max_overlap = counter
            best_sense = sense_id
    if log:
        print(" ".join(sentence))
        print("Best sense:", best_sense)
        print("{} words overlap: {}.\n".format(max_overlap, ", ".join(max_overlap_words)))
    return best_sense

In [148]:
results = []
for d in test_sentences:
    s = tokenize_uk.tokenize_text(d)[0][0]
    sense = classify_wc_idf(s, word, signatures, log=True)
    results.append(sense)
    
correct = 0
for i in range(50):
    if results[i] == correct_senses[i]:
        correct += 1
    else:
        print(test_sentences[i])
        print("\nFound sense:", results[i])
        print("\nCorrect sense:", correct_senses[i])
        print("\n============")
correct = correct*2
print(correct)

Вбігає зразу п'ять здоровенних парубків , вони несуть мішки , повні масла , а в зубах у кожного стирчить по цілій пачці .
Best sense: 1
0.3333333333333333 words overlap: масло.

Зверху , прямо на маслі , лежить іще один парубок , він блює маслом вниз .
Best sense: 3
1.6666666666666665 words overlap: прямо, масло.

І з тобою , діду , таке буде , якщо багацько масла з'їси .
Best sense: 3
0.6666666666666666 words overlap: бути, масло.

Будеш піддакувати , ми і тобі голову в масло засунемо .
Best sense: 1
1.3333333333333333 words overlap: масло, тобі.

За ними з куском масла в зубах повзе парубок в цегейковій шапці .
Best sense: 3
1.3333333333333333 words overlap: масло, за.

Дівчата , задеріть спідниці , масла дам ( регоче ) .
Best sense: 1
0.3333333333333333 words overlap: масло.

Одна з виховательок мітко жбурля чималий шматок масла йому в голову .
Best sense: 1
1.3333333333333333 words overlap: масло, шматок.

Входить дебела жінка з вусами , бородавками і куснем масла під пахвою .
Best